In [1]:
# import all the modules

import os
import csv
import requests
from zipfile import ZipFile
from io import BytesIO, TextIOWrapper
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pylab as plt


In [13]:
# grab zip file MHSA dataset
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/Accidents.zip')

# extract zipfile
with ZipFile(BytesIO(response.content)) as zf:
    # get our text file
    filename = zf.namelist() 

    # open text file
    with zf.open(filename[0]) as file:
        # make it a dataframe
        accidents = pd.read_csv(file, on_bad_lines='skip', 
                                encoding='ISO-8859-1', delimiter='|')


accidents.head()

C:\Users\Jessie\AppData\Local\Temp\ipykernel_47544\1900781676.py:12: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents = pd.read_csv(file, on_bad_lines='skip',


,MINE_ID,CONTROLLER_ID,CONTROLLER_NAME,OPERATOR_ID,OPERATOR_NAME,CONTRACTOR_ID,DOCUMENT_NO,SUBUNIT_CD,SUBUNIT,ACCIDENT_DT,...,DAYS_RESTRICT,DAYS_LOST,TRANS_TERM,RETURN_TO_WORK_DT,IMMED_NOTIFY_CD,IMMED_NOTIFY,INVEST_BEGIN_DT,NARRATIVE,CLOSED_DOC_NO,COAL_METAL_IND
0,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",MPH,220132310045,3,"STRIP, QUARY, OPEN PIT",08/12/2013,...,2.0,0,N,08/19/2013,?,NO VALUE FOUND,NaN,Employee says he slipped off back of fuel truc...,NaN,M
1,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",NaN,220241210019,30,MILL OPERATION/PREPARATION PLANT,04/22/2024,...,0.0,0,N,04/22/2024,?,NO VALUE FOUND,NaN,While blowing lime dust from the lime cooler a...,NaN,M
2,100009,M00024,Legacy Vulcan Corp (Form:Vulcan Materials Co),L16168,"Vulcan Construction Materials, LLC",NaN,220053250019,3,"STRIP, QUARY, OPEN PIT",11/13/2005,...,13.0,0,N,12/01/2005,?,NO VALUE FOUND,NaN,2 employees were attempting to load troughing ...,320053360009,M
3,100011,M11763,Imerys S A,L17074,Imerys Pigments LLC,NaN,220062000052,30,MILL OPERATION/PREPARATION PLANT,07/14/2006,...,0.0,0,N,07/15/2006,?,NO VALUE FOUND,NaN,Employee was pushing against pry bar. Bar slip...,NaN,M
4,100011,M11763,Imerys S A,0090005,Imerys Carbonates LLC,NaN,220090120006,30,MILL OPERATION/PREPARATION PLANT,01/08/2009,...,0.0,0,N,01/09/2009,?,NO VALUE FOUND,NaN,Employee tripped on c-channel at #6 silo. Fel...,NaN,M


## Geospatial 
##### Get address -> make them to lat and lon

In [20]:
# grab zip file for addresses
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/AddressofRecord.zip')

# extract zipfile
with ZipFile(BytesIO(response.content)) as zf:
    # get our text file
    filename = zf.namelist()

    # open text file
    with zf.open(filename[0]) as file:
        # make it a dataframe
        address = pd.read_csv(file, on_bad_lines='skip', 
                              encoding='ISO-8859-1', delimiter='|',
                              dtype={'ZIP_CD': str})

address.head()

,MINE_ID,MINE_NAME,CONTACT_TITLE,NEAREST_TOWN,BUSINESS_NAME,STREET,PO_BOX,CITY,STATE_ABBR,FIPS_STATE_CD,STATE,ZIP_CD,COUNTRY,PROVINCE,POSTAL_CD,MINE_TYPE_CD,MINE_STATUS,MINE_STATUS_DT,PRIMARY_SIC_CD,COAL_METAL_IND
0,100003,O'Neal Quarry & Mill,Operations Manager,Calera,"Lhoist North America of Alabama, LLC",2885 Hwy 31 South,NaN,Calera,AL,1.0,Alabama,35040,USA,NaN,NaN,Surface,Active,01/22/1979,Lime,M
1,100004,Brierfield Quarry,Quarry Manager,Brierfield,"Lhoist North America of Alabama, LLC",14786 Montevallo Road,NaN,Brierfield,AL,1.0,Alabama,35035,USA,NaN,NaN,Surface,Active,03/04/2003,"Crushed, Broken Limestone NEC",M
2,100005,Birmingham Plant,NaN,NaN,Allied Lime Company,NaN,36130,BIRMINGHAM,AL,1.0,Alabama,35236,USA,NaN,NaN,Surface,Abandoned,08/15/1989,"Crushed, Broken Limestone NEC",M
3,100006,Auburn Quarry,Safety Representative,Auburn,"Martin Marietta Materials, Inc.",245 Rockwood Rd,NaN,Tyrone,GA,13.0,Georgia,30290,USA,NaN,NaN,Surface,Active,09/24/1976,"Crushed, Broken Limestone NEC",M
4,100008,Landmark Plant,Operations Manager,Alabaster,Cheney Lime & Cement Company,1152 Old Hwy 31,NaN,Alabaster,AL,1.0,Alabama,35007,USA,NaN,NaN,Surface,Active,11/14/1975,Lime,M


In [4]:
!pip install pgeocode

In [21]:
# from zip code -> get lon and lat
import pgeocode

# set it to us
nomi = pgeocode.Nominatim('us')

# drop rows where zip codes are missing
address = address.dropna(subset=['ZIP_CD'])

# get zip code column into a list
zipcode = address['ZIP_CD'].astype(str).tolist()

# get lat and lon for all zip codes
locations = nomi.query_postal_code(zipcode)

# put it in the address column
address['lat'] = locations['latitude'].values
address['lon'] = locations['longitude'].values

# view dataframe
address.head()

,MINE_ID,MINE_NAME,CONTACT_TITLE,NEAREST_TOWN,BUSINESS_NAME,STREET,PO_BOX,CITY,STATE_ABBR,FIPS_STATE_CD,...,COUNTRY,PROVINCE,POSTAL_CD,MINE_TYPE_CD,MINE_STATUS,MINE_STATUS_DT,PRIMARY_SIC_CD,COAL_METAL_IND,lat,lon
0,100003,O'Neal Quarry & Mill,Operations Manager,Calera,"Lhoist North America of Alabama, LLC",2885 Hwy 31 South,NaN,Calera,AL,1.0,...,USA,NaN,NaN,Surface,Active,01/22/1979,Lime,M,33.1098,-86.7560
1,100004,Brierfield Quarry,Quarry Manager,Brierfield,"Lhoist North America of Alabama, LLC",14786 Montevallo Road,NaN,Brierfield,AL,1.0,...,USA,NaN,NaN,Surface,Active,03/04/2003,"Crushed, Broken Limestone NEC",M,33.0427,-86.9517
2,100005,Birmingham Plant,NaN,NaN,Allied Lime Company,NaN,36130,BIRMINGHAM,AL,1.0,...,USA,NaN,NaN,Surface,Abandoned,08/15/1989,"Crushed, Broken Limestone NEC",M,33.5446,-86.9292
3,100006,Auburn Quarry,Safety Representative,Auburn,"Martin Marietta Materials, Inc.",245 Rockwood Rd,NaN,Tyrone,GA,13.0,...,USA,NaN,NaN,Surface,Active,09/24/1976,"Crushed, Broken Limestone NEC",M,33.4719,-84.5914
4,100008,Landmark Plant,Operations Manager,Alabaster,Cheney Lime & Cement Company,1152 Old Hwy 31,NaN,Alabaster,AL,1.0,...,USA,NaN,NaN,Surface,Active,11/14/1975,Lime,M,33.1934,-86.7944


In [22]:
# if mine id from accidents matches with the address df 
# -> add lat and lon columns
accidents = accidents.merge(address[['MINE_ID', 'lat', 'lon']],
                            on = 'MINE_ID', how='left')
#accidents.dropna(inplace=True)
accidents.head()

,MINE_ID,CONTROLLER_ID,CONTROLLER_NAME,OPERATOR_ID,OPERATOR_NAME,CONTRACTOR_ID,DOCUMENT_NO,SUBUNIT_CD,SUBUNIT,ACCIDENT_DT,...,TRANS_TERM,RETURN_TO_WORK_DT,IMMED_NOTIFY_CD,IMMED_NOTIFY,INVEST_BEGIN_DT,NARRATIVE,CLOSED_DOC_NO,COAL_METAL_IND,lat,lon
0,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",MPH,220132310045,3,"STRIP, QUARY, OPEN PIT",08/12/2013,...,N,08/19/2013,?,NO VALUE FOUND,NaN,Employee says he slipped off back of fuel truc...,NaN,M,33.1098,-86.7560
1,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",NaN,220241210019,30,MILL OPERATION/PREPARATION PLANT,04/22/2024,...,N,04/22/2024,?,NO VALUE FOUND,NaN,While blowing lime dust from the lime cooler a...,NaN,M,33.1098,-86.7560
2,100009,M00024,Legacy Vulcan Corp (Form:Vulcan Materials Co),L16168,"Vulcan Construction Materials, LLC",NaN,220053250019,3,"STRIP, QUARY, OPEN PIT",11/13/2005,...,N,12/01/2005,?,NO VALUE FOUND,NaN,2 employees were attempting to load troughing ...,320053360009,M,33.3813,-86.7046
3,100011,M11763,Imerys S A,L17074,Imerys Pigments LLC,NaN,220062000052,30,MILL OPERATION/PREPARATION PLANT,07/14/2006,...,N,07/15/2006,?,NO VALUE FOUND,NaN,Employee was pushing against pry bar. Bar slip...,NaN,M,33.1756,-86.3574
4,100011,M11763,Imerys S A,0090005,Imerys Carbonates LLC,NaN,220090120006,30,MILL OPERATION/PREPARATION PLANT,01/08/2009,...,N,01/09/2009,?,NO VALUE FOUND,NaN,Employee tripped on c-channel at #6 silo. Fel...,NaN,M,33.1756,-86.3574


In [32]:
# create a new geo dataframe
import geopandas as gpd
accidents_gdf = gpd.GeoDataFrame(accidents,
                                 geometry=gpd.points_from_xy(accidents['lon'],
                                                             accidents['lat'])).set_crs("EPSG:4326", allow_override=True)
#accidents_gdf.plot(alpha=0.5, markersize=1)
#accidents_gdf.head()
# remove empty geometries
accidents_gdf = accidents_gdf[~accidents_gdf.geometry.is_empty]

In [ ]:
from folium import Choropleth
# create a map of us
us_map = folium.Map(location=[37.0902, -95.7129], 
                    tiles='cartofbpositron',
                    zoom_start=4)

# add 


In [ ]:
import folium
from folium import Marker
from folium import CircleMarker
from folium.plugins import MarkerCluster

# create a map of us
us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# get the satelite map from esri 
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(us_map)

# add points to the map
for idx, row in accidents_gdf.to_crs(epsg=4326).iterrows():
    Marker([row.geometry.y, row.geometry.x]).add_to(us_map)

us_map